In [1]:
import os

# choose working directory
os.chdir("../distances/")

os.listdir()

['distance_matrix.xlsx',
 'distancias_cidades_capitais_milhas.xlsx',
 'states_centers.csv']

In [2]:
from geopy.distance import geodesic

import pandas as pd


CIRCUITY_FACTOR = 1.205

In [3]:
# load your data
df = pd.read_csv("states_centers.csv")

df

,STATEFP,STNAME,POPULATION,LATITUDE,LONGITUDE,ST2
0,1,Alabama,5024279,33.016191,-86.753353,AL
1,4,Arizona,7151502,33.371388,-111.882468,AZ
2,5,Arkansas,3011524,35.199251,-92.713212,AR
3,6,California,39538223,35.491035,-119.347852,CA
4,8,Colorado,5773714,39.534747,-105.185361,CO
5,9,Connecticut,3605944,41.492835,-72.878714,CT
6,10,Delaware,989948,39.333614,-75.547709,DE
7,12,Florida,21538187,27.839295,-81.636016,FL
8,13,Georgia,10711908,33.410677,-83.891248,GA
9,16,Idaho,1839106,44.220476,-115.224610,ID


In [4]:
# build dict: key = ST2, value = (LATITUDE, LONGITUDE)
state_coords = dict(zip(df["ST2"], zip(df["LATITUDE"], df["LONGITUDE"])))

# example usage
print(state_coords["CA"])  # -> (35.491035, -119.347852)


df2 = pd.read_excel("../misc/tabelas.xlsx", sheet_name="table1")
df3 = pd.read_excel("../misc/tabelas.xlsx", sheet_name="tableb")

warehouse_coords = dict(
    zip(df2["Location (city)"], zip(df2["Latitude"], df2["Longitude"]))
)

print(warehouse_coords["Chicago"])  # -> (41.8781, -87.6298)

plants_coords = dict(zip(df3["Plant"], zip(df3["Latitude"], df3["Longitude"])))
print(plants_coords["New York"])  # -> (40.7128, -74.006)

(35.491035, -119.347852)
(41.8781, -87.6298)
(40.7128, -74.006)


In [5]:
df_plants = pd.DataFrame(
    index=warehouse_coords.keys(), columns=plants_coords.keys(), dtype=float
)
for plant, p_coord in plants_coords.items():
    for wh, w_coord in warehouse_coords.items():
        df_plants.at[wh, plant] = geodesic(p_coord, w_coord).miles * CIRCUITY_FACTOR
df_plants.index.name = "Warehouse Location"

In [6]:
df_states = pd.DataFrame(
    index=state_coords.keys(), columns=warehouse_coords.keys(), dtype=float
)
for state, s_coord in state_coords.items():
    for wh, w_coord in warehouse_coords.items():
        df_states.at[state, wh] = geodesic(s_coord, w_coord).miles * CIRCUITY_FACTOR
df_states.index.name = "State"

In [7]:
df_p_d = pd.DataFrame(
    index=state_coords.keys(), columns=plants_coords.keys(), dtype=float
)
for plant, p_coord in plants_coords.items():
    for state, s_coord in state_coords.items():
        df_p_d.at[state, plant] = geodesic(p_coord, s_coord).miles * CIRCUITY_FACTOR
df_p_d.index.name = "State"

In [8]:
with pd.ExcelWriter("distance_matrix.xlsx", engine="openpyxl") as writer:
    df_plants.to_excel(writer, sheet_name="d_pw")
    df_states.to_excel(writer, sheet_name="d_ws")
    df_p_d.to_excel(writer, sheet_name="d_pd")